Kiểm tra credit còn dư : https://platform.openai.com/settings/organization/billing/overview

## Chạy proj trên ubuntu <br>

Cài đặt wsl bằng cmd : `wsl --install -d Ubuntu` <br>
Nâng cấp Ubuntu từ WSL 1 lên WSL 2 : </br>
`wsl --set-version Ubuntu 2` <br>
Kiểm tra phiên bản WSL bằng lệnh:  `wsl -l -v` (wsl --list --verbose) </br>
Đặt WSL 2 làm phiên bản mặc định cho các bản phân phối Linux mớ : ` wsl --set-default-version 2` </br>
Cài đặt extension Remote explore cho VSC </br>

Khởi chạy wsl : `wsl`
Vào folder /home/TuanKiet của ubuntu , chạy git bash `git clone https://github.com/ntp2003/chatbot-tlcn`  <br>
Mở Remote explore trên VSC mở  tới /home/TuanKiet/chatbot-tlcn </br>

Update apt và cài đặt pip : `sudo apt update && sudo apt upgrade -y` </br>
`sudo apt install python3-pip -y` </br>

### Tạo môi trường ảo với Python 3

`sudo apt install python3.12-venv -y` 

`python3 -m venv .venv`

Kích hoạt môi trường ảo
- Windows: .venv\Scripts\activate
- Linux/Mac : `source .venv/bin/activate`

### Cài đặt dependencies từ pyproject.toml
#### Cài đặt Poetry  trong venv
- Window : `pip install poetry`
- Linux/macOS : `curl -sSL https://install.python-poetry.org | python3 -`

Thêm Poetry vào PATH nếu cần : `export PATH="$HOME/.local/bin:$PATH"`

#### Cài đặt môi trường dự án
Di chuyển vào thư mục chứa pyproject.toml, chạy: `poetry install`

### Kích hoạt môi trường ảo của Poetry
`poetry shell`

Sau khi bật docker image db và upgrade migration 

Khởi chạy app.py : $ chainlit run app.py -w 


Database : Postgresql với file docker-compse.yml , Alembic

- Khởi động : `docker-compose up -d` </br>
    Tự động load .env trong cùng root dir </br>
- Chạy các migrations chưa được áp dụng, đưa db lên phiên bản mới nhất: </br>
    `alembic upgrade head` </br>
- Dừng services : docker-compose down

- Tham khảo viết docker-compose.yml
    - Docker Compose file reference: https://docs.docker.com/compose/compose-file/
    - PostgreSQL configuration: https://www.postgresql.org/docs/current/runtime-config.html
    - Redis configuration: https://redis.io/topics/config

### Vị trí lưu file
Toàn bộ hệ thống file của WSL, bao gồm cả /home/TuanKiet/, được lưu trong file ảo </br> 
`C:\Users\TuanKiet\AppData\Local\Packages\CanonicalGroupLimited.Ubuntu_79rhkp1fndgsc\LocalState\ext4.vhdx` </br>

Các gói Python được cài vào:
`/usr/lib/python3/dist-packages/` </br>
Gói cài bằng pip (cho toàn hệ thống):
`/usr/local/lib/python3.*/dist-packages/` </br>
Gói cài bằng pip (cho user, nếu dùng pip install --user):
`/home/TuanKiet/.local/lib/python3.*/site-packages/` </br>

## Crawl data from fptshop.com.vn

Truy cập fptshop.com.vn và chọn danh mục như Laptop hay điện thoại 

VD với điện thoại :

Sau khi chọn danh mục điện thoại tới url `https://fptshop.com.vn/dien-thoai`

Mở devtool chọn Network -> bật filter `Fetch/XHR (XMLHttpRequest)`. Đây là các AJAX request (hoặc API request) mà trang web gửi đi khi tải dữ liệu.

Tìm POST request liên quan đến sản phẩm (ở đây là `category`)

Để chắc chắn click vào request mở phần Preview để xem data 

Lấy các thông tin cần thiết cho việc crawl ở phần Headers:
- request URL - chính là post_url cần truyền vào
- request headr

        





Kiến trúc : </br>

- Tầng model :
    Định nghĩa cấu trúc dữ liệu và tương tác db thông qua ORM (Object-Relational Mapping)- SQLAlchemy.

    2 loại model:
    - class Phone(Base) : SQLAlchemy model - interact directly with db
    - class PhoneModel(BaseModel): Pydantic model to Validate n Serialize (convert object/data into stored or transmitted such as JSON,XML,binary stream) /Deserialze data 
    

- Tầng repositories : Các hàm thao tác với data 
    Xử lý thao tác CRUD với db
    Method để truy vấn data
    Quản lý transaction và session db

Service layer (store_chatbot.py , openai) -> Repository layer (phone.py, brand.py) -> Model layer (Phone , Brand, FAQ..) -> DB


trong app.py : </br>
Khởi tạo:
- LiteralClient : Monitor OpenAI API calls , log các API calls vào hệ thống
- Redis Worker: Khởi động  Redis queue worker với scheduler để xử lý các tác vụ bất đồng bộ và hàng đợi
- Database Migration: Alembic chạy các migrations , đảm bảo schema database được cập nhật

Các module cần thiết:
    - chainlit_process.authentication: xử lý xác thực người dùng
    - chainlit_process.message: xử lý tin nhắn chat 

Khi user gửi message lên UI , Chainlit sẽ trigger decorator @cl.on_message trong chainlit_process.message.py : </br>
Hàm main() trong message.py được gọi -> gen_answer() trong service.store_chatbot.py được gọi để sinh response gửi đến UI qua send() ->
-> gọi gen_answer() trong service.store_chatbot.py -> gen_openai_answer() hay gen_answer() trong service.openai.py được gọi sinh trả lời.

gen_answer() trong service.store_chatbot.py :
- Khởi tạo messages với system prompt
- Format các history message trên Chainlit UI vào formatted_messages để gửi đến OpenAI API
- Thêm các instruction prompt như constraint, workflow, initialization vào formatted_messages
- Gọi đến gen_openai_answer() hay gen_answer() trong service.openai.py để sinh response với formatted_messages và các defined tools : faq_tool, collect_requirement_tool, search_phone_database_tool, collect_user_contact_info_tool
- Trả về Message object với content là response_text từ gen_openai_answer() hay gen_answer() trong service.openai.py

gen_openai_answer() hay gen_answer() trong service.openai.py :
- Gọi đến OpenAI API với messages, tools, model, temperature, timeout
- OpenAI sẽ trả về response với tool_calls nếu có
    - Nếu không có tool_calls hay tool_choices trả về response.content và temporary_memory và kết thúc
    - Nếu có tool_calls hay tool_choices và counter < max_iterator = 5 :
        - append model's function call message's copy vào messages
        - Gọi đến invoke_tool.py để exec tool và truyền vào tool_response content
        - append tool_response là 1 ChatCompletionToolMessageParam vào messages
        - Gọi đến OpenAI API với messages, tools, model, temperature, timeout
        - OpenAI sẽ trả về response với tool_calls nếu có
        - counter += 1

chatbot giả lập bán hàng dùng dữ liệu từ fptshop.vn, xây dựng trên nền tảng Chainlit và tích hợp với OpenAI API. Chatbot có khả năng trả lời các câu hỏi về FAQ và thông tin điện thoại từ người dùng, với dữ liệu được lưu trữ trong PostgreSQL container (docker).

## Kiến trúc tổng quan

Ứng dụng được tổ chức theo mô hình repository pattern với các thành phần chính:

- Giao diện người dùng: Xây dựng bằng Chainlit

- Cơ sở dữ liệu: 
    - PostgreSQL (trong container Docker):
        - Chứa thông tin về sản phẩm điện thoại (phones), FAQ (faqs), thương hiệu (brands), và thông tin người dùng (users).
        - Dữ liệu được truy vấn thông qua SQLAlchemy ORM.
    - Redis:
        - Lưu trữ tạm thời số trang (paging) khi truy vấn điện thoại vào cache, giúp quản lý phân trang và trạng thái cuộc trò chuyện.
    - Alembic:
        - Đảm bảo quản lý phiên bản của cơ sở dữ liệu.


- Xử lý ngôn ngữ tự nhiên: Sử dụng OpenAI API


## Quy trình hoạt động

1. Khởi động ứng dụng:

- File app.py là điểm vào của ứng dụng, được chạy bằng lệnh `chainlit run app.py -w`

- Khi khởi động, ứng dụng chạy migration với Alembic để cập nhật cấu trúc database

- Khởi tạo Redis worker để xử lý các tác vụ bất đồng bộ

2. Xác thực người dùng:

Người dùng đăng nhập qua giao diện Chainlit

Hàm `auth_callback` trong `chainlit_process/authentication.py` xử lý xác thực

Thông tin người dùng được kiểm tra trong database

3. Xử lý tin nhắn:

Khi người dùng gửi tin nhắn, hàm main trong `chainlit_process/message.py` được gọi

Tin nhắn được chuyển đến `service/store_chatbot.py` để xử lý

4. Tạo phản hồi:

Hàm `gen_answer` trong `service/store_chatbot.py` xử lý tin nhắn từ người dùng

Các prompt hệ thống được thêm vào để định hình vai trò và hành vi của chatbot

Tin nhắn được gửi đến OpenAI API thông qua `service/openai.py`

5. Sử dụng công cụ (tools):

Chatbot dùng các tools để thực hiện các tác vụ cụ thể:

`collect_requirement_tool`: Thu thập yêu cầu của người dùng

`search_phone_database_tool`: Tìm kiếm thông tin điện thoại

`faq_tool`: Trả lời câu hỏi thường gặp

`collect_user_contact_info_tool`: Thu thập thông tin liên hệ

6. Lưu trữ bộ nhớ người dùng:

Thông tin về nhu cầu, sở thích và yêu cầu của người dùng được lưu trong user_memory

Dữ liệu này được sử dụng để cá nhân hóa trải nghiệm và đưa ra gợi ý phù hợp

7. Tìm kiếm thông tin điện thoại:

Sử dụng vector embedding để tìm kiếm ngữ nghĩa

Hỗ trợ tìm kiếm theo tên, thương hiệu, khoảng giá

8. Trả lời câu hỏi thường gặp:

Sử dụng vector embedding để tìm câu hỏi tương tự trong cơ sở dữ liệu FAQ

Trả về câu trả lời phù hợp nhất

### Các thành phần chính
#### Models
`User`: Lưu trữ thông tin người dùng

`UserMemory`: Lưu trữ thông tin về nhu cầu và sở thích của người dùng

`Phone`: Lưu trữ thông tin về điện thoại

`Laptop`: Lưu trữ thông tin về Laptop

`Brand`: Lưu trữ thông tin về thương hiệu

`FAQ`: Lưu trữ câu hỏi thường gặp và câu trả lời

#### Repositories
Các repository chịu trách nhiệm tương tác với cơ sở dữ liệu:

`user.py`: Quản lý người dùng

`user_memory.py`: Quản lý bộ nhớ người dùng

`phone.py`: Quản lý thông tin điện thoại

`brand.py`: Quản lý thương hiệu

`faq.py`: Quản lý FAQ

#### Services
`openai.py`: Tương tác với OpenAI API

`embedding.py`: Tạo vector embedding

`store_chatbot.py`: Xử lý tin nhắn và tạo phản hồi

`converter.py`: Chuyển đổi dữ liệu

`email.py`: Gửi email

#### Tools
Các công cụ được sử dụng bởi chatbot:

- `collect_requirement.py`: Thu thập yêu cầu của người dùng

- `search_phone_database.py`: Tìm kiếm thông tin điện thoại

- `faq.py`: Trả lời câu hỏi thường gặp

- `collect_user_contact_info.py`: Thu thập thông tin liên hệ

- `invoke_tool.py`: Gọi các công cụ khác

#### Quy trình xử lý tin nhắn chi tiết
1. Người dùng gửi tin nhắn qua giao diện Chainlit

2. Tin nhắn được chuyển đến hàm main trong chainlit_process/message.py

3. Hàm `gen_answer` trong `service/store_chatbot.py` được gọi với ID người dùng, ID cuộc trò chuyện và lịch sử tin nhắn

4. Các prompt system định hình vai trò và hành vi của chatbot

5. Tin nhắn được gửi đến OpenAI API thông qua service/openai.py

6. OpenAI API trả về phản hồi hoặc yêu cầu sử dụng công cụ

7. Nếu cần dùng function call tool, `invoke_tool` được gọi để thực hiện tác vụ

Kết quả từ công cụ được gửi lại cho OpenAI API để tạo phản hồi cuối cùng

Phản hồi cuối cùng được gửi lại cho người dùng qua giao diện Chainlit

Chatbot này đã được thiết kế để mô phỏng nhân viên tư vấn bán hàng chuyên nghiệp, với khả năng trả lời các câu hỏi về chính sách cửa hàng, tìm kiếm và đề xuất sản phẩm phù hợp với nhu cầu của người dùng, và thu thập thông tin liên hệ để hỗ trợ sau bán hàng.

1. Tạo fanpage facebook Chatbot Ecommerce
2. Vào https://developers.facebook.com/ -> CreateApp
3. Generate AccessToken 

Triển khai webhook
1. Chạy ứng dụng FastAPI: -> 127.0.0.1:8000
    `uvicorn messenger:app --reload`

2. Sử dụng ngrok để tạo URL công khai:
    `ngrok http 8000`

3. Cấu hình webhook trong Facebook Developer:
- Đi đến trang Facebook Developer
- Chọn ứng dụng của bạn
- Vào phần Messenger > Settings
- Trong phần Webhooks, nhấp vào "Add Callback URL"
- Nhập URL ngrok của bạn (ví dụ: https://your-ngrok-url.ngrok.io/webhook)
- Nhập Verify Token giống với FB_VERIFY_TOKEN trong file .env
- Chọn các sự kiện bạn muốn nhận (ít nhất là "messages")

Lưu ý quan trọng
1. Đảm bảo đã cấp quyền pages_messaging cho ứng dụng Facebook .
2. Kết nối ứng dụng với Trang Facebook của bạn để nhận Page Access Token.
3. Trong môi trường sản xuất, bạn nên lưu trữ lịch sử trò chuyện trong cơ sở dữ liệu thay vì biến trong bộ nhớ.
4. Xử lý các loại tin nhắn khác nhau (hình ảnh, nút, v.v.) tùy theo nhu cầu .

Với webhook này, ứng dụng chatbot của bạn sẽ có thể nhận và phản hồi tin nhắn từ người dùng thông qua Facebook Messenger.

Nếu tích hợp chatbot này lên facebook page để người dùng facebook chat với bot thông qua page thì nên xử lý thread_id này như thế nào ? Nên tạo thread_id random bằng uuid4() hay get conversation_id từ facebook qua API graph của facebook hay tạo thông qua sender_id kiểu persistent_thread_id = uuid.uuid5(uuid.NAMESPACE_DNS, sender_id) ? Vì không thể tạo nhiều nhiều session như chainlit . Nếu tạo random thread_id thì lần sau khi user chat lại với bot trên facebook page sẽ tạo mới 1 thread_id không thể giữ được các context trò chuyện trước đây

1. Phân tích các lựa chọn
Tạo thread_id ngẫu nhiên (uuid4()): Cách này không phù hợp vì mỗi lần người dùng nhắn lại (hoặc server khởi động lại, mất state in-memory), một thread_id mới sẽ được tạo, dẫn đến mất liên kết với user_memory cũ. Bot sẽ không nhớ gì về cuộc trò chuyện trước đó. => Không nên dùng.

2. Lấy conversation_id từ Graph API:
- Ưu điểm: Facebook cung cấp một ID ổn định cho cuộc trò chuyện giữa người dùng và Trang (conversation_id). ID này là duy nhất và bền vững, phản ánh đúng bản chất "luồng" trò chuyện trên Messenger.
Nhược điểm:
- Thêm API Call: Yêu cầu gọi thêm một API (endpoint /conversations) cho mỗi lần xử lý tin nhắn (hoặc ít nhất là lần đầu), làm tăng độ trễ và có thể gặp giới hạn tỷ lệ (rate limiting) của Facebook.
Phụ thuộc: Phụ thuộc vào tính sẵn có và tốc độ phản hồi của Graph API.
- Fallback không ổn định: Như trong code ví dụ get_conversation_id, nếu không lấy được conversation_id, fallback về f"{sender_id}_{int(time.time())}" cũng không giải quyết được vấn đề duy trì context vì nó vẫn thay đổi theo thời gian.

3. Sử dụng sender_id làm cơ sở cho thread_id:

- Phân tích: sender_id (Page-Scoped User ID - PSID) là định danh duy nhất và ổn định cho người dùng tương tác với Trang của bạn. Đây chính là yếu tố cốt lõi để xác định ai đang nói chuyện. Về bản chất, toàn bộ lịch sử chat của một người dùng với page của bạn chính là một "luồng" (thread) duy nhất.
Ưu điểm:
- Luôn có sẵn: sender_id luôn có trong payload webhook, không cần gọi thêm API.
- Ổn định: Đảm bảo cùng một người dùng luôn được liên kết với cùng một lịch sử/bộ nhớ.
- Đơn giản: Giảm sự phức tạp so với việc gọi thêm API.
- Nhược điểm/Cần điều chỉnh:
Hàm gen_answer và hệ thống user_memory của bạn đang mong đợi một thread_id (có thể là kiểu UUID từ Chainlit). Bạn cần điều chỉnh để hệ thống user_memory sử dụng một định danh ổn định dựa trên sender_id.

Chi tiết luồng xử lý:

```mermaid
graph TD
    subgraph Chainlit Process
        A[Chainlit UI] --> B[message.py]
        B --> C[authentication.py]
        B --> D[data_layer.py]
    end

    subgraph Main Flow
        D --> E[store_chatbot_v2.py]
        E --> F[gen_answer]
    end

    subgraph Detect Demand
        F --> G[detect_demand.py]
        G --> H{Product Type?}
        H -->|Mobile Phone| I[Handle Phone]
        H -->|Laptop| J[Handle Laptop]
        H -->|Accessories| K[Handle Accessories]
        H -->|Undetermined| L[Handle Undetermined]
    end

    subgraph Phone Flow
        I --> M[phone/collect_and_retrieval.py]
        M --> N[Update User Memory]
        N --> O[phone/generate_response.py]
    end

    subgraph Laptop Flow
        J --> P[laptop/collect_and_retrieval.py]
        P --> Q[Update User Memory]
        Q --> R[laptop/generate_response.py]
    end

    subgraph Accessories Flow
        K --> S[accessories/collect_and_retrieval.py]
        S --> T[Update User Memory]
        T --> U[accessories/generate_response.py]
    end

    subgraph Undetermined Flow
        L --> V[undetermined/generate_response.py]
    end

    subgraph Utils & Memory
        W[utils.py]
        X[User Memory DB]
        Y[Redis Cache]
    end

    %% Connections to Utils & Memory
    M --> W
    O --> W
    P --> W
    R --> W
    S --> W
    U --> W
    V --> W
    G --> W

    N --> X
    Q --> X
    T --> X
    V --> X

    N --> Y
    Q --> Y
    T --> Y

    %% Return Flow
    O --> |Response| F
    R --> |Response| F
    U --> |Response| F
    V --> |Response| F
```

1. Chainlit Process:
- Bắt đầu từ Chainlit UI
- Xử lý message thông qua message.py
- Kiểm tra authentication qua authentication.py
- Lưu trữ data qua data_layer.py

2. Main Flow:
- store_chatbot_v2.py nhận request và gọi hàm gen_answer
- Xử lý conversation history và user memory

3. Detect Demand:
- detect_demand.py phân tích nhu cầu người dùng
- Phân loại thành 4 loại: Mobile Phone, Laptop, Accessories, hoặc Undetermined

4. Product Specific Flows:

- Phone Flow:
    - Collect và retrieval thông tin điện thoại
    - Update user memory
    - Generate response về điện thoại
- Laptop Flow:
    - Collect và retrieval thông tin laptop
    - Update user memory
    - Generate response về laptop
- Accessories Flow:
    - Collect và retrieval thông tin phụ kiện
    - Update user memory
    - Generate response về phụ kiện
- Undetermined Flow:
    - Generate response chung khi không xác định được nhu cầu cụ thể
5. Utils & Memory:
- utils.py cung cấp các utility functions chung
- User Memory DB lưu trữ thông tin người dùng
- Redis Cache lưu trữ temporary data và offset

Mỗi luồng xử lý đều:
- Sử dụng các utility functions từ utils.py
- Tương tác với User Memory DB để lưu/đọc thông tin
- Sử dụng Redis để cache data tạm thời
- Trả về response cho người dùng thông qua main flow

1. Khởi đầu cuộc hội thoại (Entry Point):

```python
# chainlit_process/message.py
@cl.on_message
async def main(message: cl.Message):
```
Khi người dùng gửi tin nhắn, Chainlit framework sẽ trigger event này </br>
Kiểm tra đăng nhập và lấy thông tin user_id, thread_id </br>
Tạo/lấy thread cho cuộc hội thoại </br>

2. Xử lý tin nhắn và tạo phản hồi:

```python
# service/store_chatbot_v2.py
def gen_answer(user_id, thread_id, history):
```
Luồng xử lý chính gồm 3 giai đoạn:

**Giai đoạn 1: Phát hiện nhu cầu**

```python
# agents/detect_demand.py
class Agent(AgentBase):
    def run(self, messages):
        # 1. Phân tích nội dung tin nhắn
        # 2. Xác định loại nhu cầu (điện thoại/chưa xác định)
        # 3. Thu thập thông tin liên hệ nếu có
```

- Sử dụng OpenAI API để phân tích nội dung
- Cập nhật UserMemory với loại nhu cầu và thông tin liên hệ
- Nếu phát hiện thông tin liên hệ mới → gửi email thông báo

**Giai đoạn 2: Thu thập thông tin chi tiết**

Nếu là nhu cầu về điện thoại:

```python
# agents/phone/collect_and_retrieval.py
class Agent(AgentBase):
    def run(self, messages):
        # 1. Phân tích yêu cầu về điện thoại
        # 2. Thu thập thông tin về:
        #    - Giá cả mong muốn
        #    - Thương hiệu ưa thích
        #    - Các yêu cầu đặc biệt
        # 3. Tìm kiếm sản phẩm phù hợp
```

**Giai đoạn 3: Tạo câu trả lời**

Dựa vào loại nhu cầu:

a) Nhu cầu về điện thoại:
```python
# agents/phone/generate_response.py
class Agent(AgentBase):
    def run(self, conversation_messages, instructions, phone_knowledge):
        # 1. Tổng hợp thông tin đã thu thập
        # 2. Tạo câu trả lời phù hợp
        # 3. Đề xuất sản phẩm nếu có
```

b) Nhu cầu chưa xác định:

```python
# agents/undetermined/generate_response.py
class Agent(AgentBase):
    def run(self, conversation_messages):
        # Tạo câu trả lời chung và hỏi thêm về nhu cầu
```

3. Quản lý bộ nhớ và trạng thái:

```python
# models/user_memory.py
class UserMemory(Base):
    # Lưu trữ:
    # - Thông tin người dùng
    # - Nhu cầu hiện tại
    # - Thông tin liên hệ
    # - Trạng thái tư vấn
    # - Bộ lọc tìm kiếm
```

4. Lưu trữ dữ liệu

```python
# repositories/user_memory.py
def create(data: CreateUserMemoryModel)
def update(id: UUID, data: UpdateUserMemoryModel)
def get_by_thread_id(thread_id: UUID)
```
5. Xử lý tin nhắn và phản hồi:

```python
# chainlit_process/message.py
# 1. Lưu tin nhắn người dùng
message_data = CreateMessageModel(...)
new_message = create_message(message_data)

# 2. Tạo và lưu phản hồi
new_assistant_message = create_message(new_assistant_message_data)

# 3. Gửi phản hồi
return await cl.Message(...).send()
```
6. Các tính năng phụ trợ:

a) xử lý email:
```python
# service/email.py
def create_message(sender, to, subject, message_text)
def send_message(message)
```

b) Chuyển đổi dữ liệu:
```python
# service/converter.py
def convert_to_standard_phone_number(phone_number)
def convert_to_standard_email(email)
```

7. Luồng dữ liệu tổng thể:

```plaintext
graph TD
    A[User Message] --> B[Chainlit Framework]
    B --> C[Message Handler]
    C --> D[Store Chatbot]
    D --> E[Detect Demand Agent]
    E --> F{Demand Type}
    F -->|Phone| G[Phone Agent]
    F -->|Undetermined| H[Undetermined Agent]
    G --> I[Generate Response]
    H --> I
    I --> J[Update User Memory]
    J --> K[Send Response]
```

Workflow tổng thể:
- Nhận tin nhắn từ người dùng
- Phân tích nhu cầu và thu thập thông tin
- Tìm kiếm thông tin sản phẩm phù hợp (nếu có)
- Tạo câu trả lời phù hợp
- Cập nhật trạng thái và bộ nhớ
- Gửi phản hồi cho người dùng

1. 120a604fd902_add_vector_extension.py 
- Thêm vector extension cho PostgreSQL
- Bước đầu tiên để hỗ trợ vector search

2. dc9bc54eaed6_create_phone_table.py
- Tạo bảng phones ban đầu

3. 9e5da9e02a91_create_users_table.py
- Tạo bảng users với các trường cơ bản

4. 672acb86aec9_create_faqs_table.py
- Tạo bảng faqs cho câu hỏi thường gặp

5. 0b47af67e745_edit_phone_table.py
- Chỉnh sửa cấu trúc bảng phones

6. dd89958929b9_set_contraint_not_null_to_phone_table.py
- Thêm ràng buộc NOT NULL cho các cột trong bảng phones

7. f4873b4eba7e_create_brand_table.py
- Tạo bảng brands để lưu thông tin thương hiệu

8. 806ac93114b7_modify_brand_code_column_to_phones.py
- Sửa đổi cột brand_code trong bảng phones

9. 45d35c46a219_add_user_memory_table.py
- Tạo bảng user_memory lưu context người dùng

10. 71321ac3cff1_add_embedding_column_for_phone_name_.py
- Thêm cột embedding cho tên điện thoại

11. 0113278eac18_modify_name_embedding_column_of_phones_.py
- Sửa đổi cột name_embedding trong bảng phones

12. 535089ddf724_add_contact_info_columns_into_user_.py
- Thêm contact info (mail,phone) vào bảng user_memory

13. 888cecf62e00_create_threads_table.py
- Tạo bảng threads cho các cuộc hội thoại

14. df9b9ff692d4_create_messages_table.py
- Tạo bảng messages cho tin nhắn trong thread

15. 2f57a79a7d35_rename_brand_to_brand_code_and_add_.py
- Đổi tên cột brand thành brand_code
- Thêm các trường mới

16. b0b9ba1f513b_add_intent_column_to_user_memory.py
- Thêm cột intent vào bảng user_memory

17. ee9837b95476_add_current_filter_and_consultation_.py
- Thêm current_filter và consultation_status vào user_memory

18. 4693c87a62f4_create_laptop_table.py
- Tạo bảng laptops với cấu trúc tương tự phones

19. 6dbf4daf29c5_create_accessory_table.py
- Tạo bảng accessories với cấu trúc tương tự phones

20. 51eafed45404_edit_user_table.py
- Chỉnh sửa bảng users, thêm fb_id, gender field

21. 















    users {
        uuid id PK
        string user_name
        string password_hash
        string email
        string phone_number
        string full_name
        string avatar_url
        timestamp created_at
        timestamp updated_at
    }

    threads {
        uuid id PK
        uuid user_id FK
        string name
        timestamp created_at
        timestamp updated_at
    }

    messages {
        uuid id PK
        uuid thread_id FK
        string type
        text content
        timestamp created_at
        timestamp updated_at
    }

    user_memory {
        uuid id PK
        uuid user_id FK
        uuid thread_id FK
        text user_demand
        text product_name
        text brand_code
        text brand_name
        integer min_price
        float max_price
        text phone_number
        text email
        jsonb intent
        jsonb current_filter
        jsonb consultation_status
        timestamp created_at
        timestamp updated_at
    }

    phones {
        string id PK
        string name
        string slug
        string brand_code FK
        string product_type
        text description
        jsonb promotions
        jsonb skus
        jsonb key_selling_points
        float price
        float score
        jsonb data
        vector name_embedding
        timestamp created_at
        timestamp updated_at
    }

    laptops {
        string id PK
        string name
        string slug
        string brand_code FK
        string product_type
        text description
        jsonb promotions
        jsonb skus
        jsonb key_selling_points
        float price
        float score
        jsonb data
        vector name_embedding
        timestamp created_at
        timestamp updated_at
    }

    brands {
        string id PK
        string name
        string slug
        jsonb data
        timestamp created_at
        timestamp updated_at
    }

    faqs {
        uuid id PK
        string question
        string answer
        vector question_embedding
        timestamp created_at
        timestamp updated_at
    }

    users ||--o{ threads : "has"
    threads ||--o{ messages : "contains"
    users ||--o{ user_memory : "has"
    threads ||--o{ user_memory : "has"
    brands ||--o{ phones : "has"
    brands ||--o{ laptops : "has"

```python
from openai.types.chat import (
    ChatCompletionMessageParam,
    ChatCompletionToolMessageParam,
    ChatCompletionToolParam,
) # typehint to ensure safety

messages: list[ChatCompletionMessageParam]
tools: Iterable[ChatCompletionToolParam],


response = (
        _client.chat.completions.create(
            messages=messages,
            model=model,
            user=str(user_id),
            tools=tools,
            temperature=0,
            timeout=30,
        )
        .choices[0]
        .message
    )
    # OpenAI will return tool_calls corresponding to the tools used by the model with instructions prompt
    tool_choices = response.tool_calls
    # return response immediately if no tool_calls
    if not tool_choices:
        if not response.content:
            raise Exception("No response content from the model")
        print("Final response:", response.content)
        return response.content, temporary_memory

    while tool_choices and counter < max_iterator:
        counter += 1
        
        print("counter:", counter) 
        # append model's function call response message's copy
        messages.append(response.model_copy())  # type: ignore
        print("\n\n==== tool_calls ====\n\n")
        for tool in tool_choices:
            print(tool.function.name)
            call_id = tool.id
            tool_name = tool.function.name
            args = json.loads(tool.function.arguments) # parse tool func arguments to json
            tool_response: ChatCompletionToolMessageParam = {
                "role": "tool",
                "tool_call_id": call_id,
                "content": invoke(user_id, thread_id, tool_name, args), # get tool response from invoke_tool.py
            }
            # append tool result to messages
            messages.append(tool_response)  # type: ignore
            # store tool result to temporary memory
            temporary_memory[tool_name] = tool_response["content"]
            print("Tool response:", tool_response["content"])
        response = (
            _client.chat.completions.create(
                messages=messages,
                model=model,
                user=str(user_id),
                tools=tools,
                temperature=0,
                timeout=30,
            )
            .choices[0]
            .message
        )
        tool_choices = response.tool_calls

    if counter == max_iterator:
        raise Exception(
            f"Maximum iteration reached ({max_iterator}). Please try again."
        )
    if not response.content:
        raise Exception("No response content from the model")

    print("Final response:", response.content)
    return response.content, temporary_memory
```

```mermaid
classDiagram
    class BaseFilter {
        +Config config
        +str brand_code
        +float max_price
        +int min_price
        +str name
        +get_price_condition_expression()
        +get_brand_condition_expression()
        +get_name_condition_expression()
        +condition_expression()
        +score_expression()
        +to_statement()
    }
    
    class PhoneFilter {
        +phone specific fields
    }
    
    class LaptopFilter {
        +str cpu
        +int ram
        +int storage
        +get_cpu_condition_expression()
        +get_ram_condition_expression()
    }
    
    class AccessoryFilter {
        +str category
        +get_category_condition_expression()
    }
    
    BaseFilter <|-- PhoneFilter
    BaseFilter <|-- LaptopFilter
    BaseFilter <|-- AccessoryFilter
```